In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
import tensorflow as tf

In [2]:
data_train = pd.read_csv('../../../data/train_data.csv').values
data_test = pd.read_csv('../../../data/test_data.csv').values

print(data_train.shape[1])

# Build X and y
X_train = data_train[:, :-1]
y_train = data_train[:, -1]
X_test = data_test[:, :-1]
y_test = data_test[:, -1]

# Number of atributes in training data
n_att = X_train.shape[1]

# Neurons
n_neurons_1 = 256
n_neurons_2 = 128
n_neurons_3 = 64
n_neurons_4 = 32

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_att])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

# Hidden weights
W_hidden_1 = tf.Variable(weight_initializer([n_att, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output weights
W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
bias_out = tf.Variable(bias_initializer([1]))

# Hidden layer
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (transpose!)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

# Cost function
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

# Saver
saver = tf.train.Saver()

15


In [3]:
with tf.Session() as sess:
    
    # Init
    sess.run(tf.global_variables_initializer())

    # Fit neural net
    batch_size = 256
    mse_train = []
    mse_test = []

    # Run
    epochs = 20
    for e in range(epochs):

        # Shuffle training data
        shuffle_indices = np.random.permutation(np.arange(len(y_train)))
        X_train = X_train[shuffle_indices]
        y_train = y_train[shuffle_indices]

        # Minibatch training
        for i in range(0, len(y_train) // batch_size):
            start = i * batch_size
            batch_x = X_train[start:start + batch_size]
            batch_y = y_train[start:start + batch_size]
            # Run optimizer with batch
            sess.run(opt, feed_dict={X: batch_x, Y: batch_y})

            # Show progress
            if np.mod(i, 50) == 0:
                # MSE train and test
                mse_train.append(sess.run(mse, feed_dict={X: X_train, Y: y_train}))
                mse_test.append(sess.run(mse, feed_dict={X: X_test, Y: y_test}))
                print('MSE Train: ', mse_train[-1])
                print('MSE Test: ', mse_test[-1])
                # Prediction
                pred = sess.run(out, feed_dict={X: X_test})
                print('pred: ', pred)
    save_path = saver.save(sess, "../models/model.ckpt")
    print("Model saved in path: %s" % save_path)

MSE Train:  0.9795074
MSE Test:  0.97684836
pred:  [[ 0.05059671 -0.04411414 -0.03971947 ...  0.11060814  0.09584098
   0.01097042]]
MSE Train:  0.10879886
MSE Test:  0.10997727
pred:  [[-0.18895778 -0.5180253  -0.45053053 ... -0.34976786 -0.28672057
   3.3590546 ]]
MSE Train:  0.06164637
MSE Test:  0.06288805
pred:  [[-0.33265814 -0.3220619  -0.31529674 ... -0.5123952  -0.323401
   3.447411  ]]
MSE Train:  0.04403743
MSE Test:  0.04430341
pred:  [[-0.44569558 -0.31615597 -0.31244007 ... -0.48193043 -0.36472607
   3.466197  ]]
MSE Train:  0.04806081
MSE Test:  0.04755624
pred:  [[-0.48032108 -0.3898917  -0.38311395 ... -0.5185114  -0.4059641
   3.112692  ]]
MSE Train:  0.023989895
MSE Test:  0.024401754
pred:  [[-0.5093992  -0.39831766 -0.40550983 ... -0.52957773 -0.43369472
   3.3342924 ]]
MSE Train:  0.025936136
MSE Test:  0.026453778
pred:  [[-0.5321434  -0.36096594 -0.39532268 ... -0.47114244 -0.3910291
   2.940453  ]]
MSE Train:  0.016634205
MSE Test:  0.017131737
pred:  [[-0.4984

MSE Train:  0.004016394
MSE Test:  0.0043175076
pred:  [[-0.50512046 -0.34928018 -0.41944963 ... -0.4059245  -0.373506
   3.4684794 ]]
MSE Train:  0.0043281303
MSE Test:  0.00461128
pred:  [[-0.49890727 -0.33624232 -0.41185826 ... -0.41837096 -0.3876065
   3.6354206 ]]
MSE Train:  0.0032221377
MSE Test:  0.0035628823
pred:  [[-0.506989   -0.37031665 -0.44080845 ... -0.4363766  -0.40152702
   3.5325947 ]]
MSE Train:  0.0029808278
MSE Test:  0.0033104233
pred:  [[-0.50618976 -0.36919397 -0.4421155  ... -0.44412327 -0.41321236
   3.6459131 ]]
MSE Train:  0.0031885053
MSE Test:  0.0034707347
pred:  [[-0.49844486 -0.36391655 -0.44042736 ... -0.44609252 -0.40832224
   3.5896375 ]]
MSE Train:  0.0032623284
MSE Test:  0.003543399
pred:  [[-0.5108715  -0.41284755 -0.45794487 ... -0.45192787 -0.42098257
   3.6567569 ]]
MSE Train:  0.0031717743
MSE Test:  0.003399561
pred:  [[-0.50658524 -0.3673403  -0.4317616  ... -0.44318995 -0.40607867
   3.7468636 ]]
MSE Train:  0.0037375551
MSE Test:  0.0040

MSE Train:  0.004495551
MSE Test:  0.004659163
pred:  [[-0.508797   -0.4534476  -0.49123678 ... -0.46972275 -0.37078902
   3.9208326 ]]
MSE Train:  0.0019182346
MSE Test:  0.0020789797
pred:  [[-0.51196593 -0.38798967 -0.45484823 ... -0.4269402  -0.3519119
   3.6473053 ]]
MSE Train:  0.0021752438
MSE Test:  0.0023285158
pred:  [[-0.5076096  -0.37908953 -0.45684835 ... -0.45071924 -0.35766008
   3.5930867 ]]
MSE Train:  0.0020146298
MSE Test:  0.0022001127
pred:  [[-0.5067424  -0.35594004 -0.44495258 ... -0.4297139  -0.36397415
   3.553907  ]]
MSE Train:  0.002030102
MSE Test:  0.002184189
pred:  [[-0.5100542  -0.3824447  -0.45723525 ... -0.43268886 -0.35009196
   3.6613352 ]]
MSE Train:  0.0019830375
MSE Test:  0.0021712594
pred:  [[-0.5076211  -0.40280062 -0.4763488  ... -0.445477   -0.36416313
   3.5071409 ]]
MSE Train:  0.002138087
MSE Test:  0.0023591623
pred:  [[-0.51736647 -0.430032   -0.48127118 ... -0.46466202 -0.35762066
   3.6569712 ]]
MSE Train:  0.0021734105
MSE Test:  0.00

MSE Train:  0.0015106882
MSE Test:  0.0016967119
pred:  [[-0.524945   -0.38962042 -0.4714815  ... -0.42692792 -0.35734686
   3.6119697 ]]
MSE Train:  0.0017930317
MSE Test:  0.0018639024
pred:  [[-0.5114418  -0.3826206  -0.4465006  ... -0.4120256  -0.33013043
   3.683849  ]]
MSE Train:  0.001399183
MSE Test:  0.001529925
pred:  [[-0.50343615 -0.38109836 -0.45240447 ... -0.43530715 -0.35931212
   3.5936298 ]]
MSE Train:  0.001530514
MSE Test:  0.0016469589
pred:  [[-0.5051583  -0.37770092 -0.4601644  ... -0.42164025 -0.33098158
   3.606781  ]]
MSE Train:  0.0014391432
MSE Test:  0.0016014372
pred:  [[-0.51008093 -0.41053367 -0.4699121  ... -0.46039322 -0.3700217
   3.6062167 ]]
MSE Train:  0.0012669321
MSE Test:  0.0013704695
pred:  [[-0.505685   -0.3807918  -0.45785594 ... -0.44208074 -0.36424956
   3.5792828 ]]
MSE Train:  0.0017747696
MSE Test:  0.0018998341
pred:  [[-0.51317173 -0.40898386 -0.45920616 ... -0.44414976 -0.33120695
   3.576294  ]]
MSE Train:  0.0015921638
MSE Test:  0.

MSE Train:  0.0014097557
MSE Test:  0.0015429249
pred:  [[-0.50860465 -0.39110926 -0.44897008 ... -0.43243492 -0.32308793
   3.6417367 ]]
MSE Train:  0.0018152748
MSE Test:  0.0019069646
pred:  [[-0.5019549  -0.3484517  -0.42779368 ... -0.42232436 -0.3158571
   3.809165  ]]
MSE Train:  0.0010689505
MSE Test:  0.0011454996
pred:  [[-0.5198527  -0.40704933 -0.45842242 ... -0.44257826 -0.3596217
   3.6544132 ]]
MSE Train:  0.0010339202
MSE Test:  0.0011645498
pred:  [[-0.5199755  -0.41038913 -0.4657791  ... -0.4462287  -0.32471392
   3.6344721 ]]
MSE Train:  0.0010562919
MSE Test:  0.0011596751
pred:  [[-0.5211413  -0.42181677 -0.46643236 ... -0.44423312 -0.325022
   3.6863    ]]
MSE Train:  0.0015024173
MSE Test:  0.0015835966
pred:  [[-0.5075569  -0.4143239  -0.45051363 ... -0.42926013 -0.33551273
   3.6745512 ]]
MSE Train:  0.0017825663
MSE Test:  0.0018745018
pred:  [[-0.5158294  -0.38664144 -0.45299634 ... -0.43733266 -0.31142253
   3.7148852 ]]
MSE Train:  0.0010766936
MSE Test:  0.

MSE Train:  0.0007567971
MSE Test:  0.0008605062
pred:  [[-0.52314925 -0.41362104 -0.45854643 ... -0.4434143  -0.33183444
   3.561757  ]]
MSE Train:  0.00074114325
MSE Test:  0.00081925705
pred:  [[-0.5159909  -0.40448734 -0.45837155 ... -0.437513   -0.32063064
   3.5490618 ]]
MSE Train:  0.0009316264
MSE Test:  0.0010078561
pred:  [[-0.50659597 -0.38897678 -0.44882587 ... -0.4391127  -0.33232227
   3.6255703 ]]
MSE Train:  0.0012384946
MSE Test:  0.0013339842
pred:  [[-0.51049703 -0.39037597 -0.45251092 ... -0.42560565 -0.3116891
   3.5270555 ]]
MSE Train:  0.0011312079
MSE Test:  0.001213234
pred:  [[-0.5117171  -0.40317312 -0.459128   ... -0.447124   -0.32708228
   3.7177107 ]]
MSE Train:  0.000798999
MSE Test:  0.0008686968
pred:  [[-0.5126721  -0.41336715 -0.46297562 ... -0.4441358  -0.331828
   3.6837695 ]]
MSE Train:  0.00093546766
MSE Test:  0.0009996883
pred:  [[-0.5078197  -0.39641953 -0.45655903 ... -0.43210214 -0.29392827
   3.6113703 ]]
MSE Train:  0.00086559524
MSE Test: 